In [36]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os

# ========================================
# 경로 설정 (여기를 수정하세요)
# ========================================
DATA_DIR = '/content/drive/MyDrive/cifar10_data'  # CIFAR-10 데이터를 저장할 경로

# 데이터 디렉토리 생성
os.makedirs(DATA_DIR, exist_ok=True)

In [37]:
# --- CIFAR-10 로드 (자동으로 캐싱) ---
# Keras는 기본적으로 ~/.keras/datasets에 저장하지만,
# 명시적으로 경로를 지정하려면 아래와 같이 처리
cifar10_path = os.path.join(DATA_DIR,'cifar-10-batches-py')

if os.path.exists(cifar10_path):
    print(f"✓ 기존 데이터 발견: {cifar10_path}")
    print("저장된 데이터를 로드합니다...")
else:
    print(f"데이터가 없습니다. {DATA_DIR}에 다운로드합니다...")

데이터가 없습니다. /content/drive/MyDrive/cifar10_data에 다운로드합니다...


In [38]:
# 데이터 로드 (없으면 자동 다운로드 후 캐싱)
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(f"✓ 데이터 로드 완료")
print(f"  - 학습 데이터: {x_train.shape}")
print(f"  - 테스트 데이터: {x_test.shape}\n")


✓ 데이터 로드 완료
  - 학습 데이터: (50000, 32, 32, 3)
  - 테스트 데이터: (10000, 32, 32, 3)



In [39]:
# 클래스 이름 정의
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [40]:
# 사용할 클래스 선택
selected_classes = ['cat','dog','horse']
selected_idx = [class_names.index(c)for c in selected_classes]

# --- 해당 클래스만 필터링 ---
train_mask = np.isin(y_train, selected_idx).flatten()
test_mask = np.isin(y_test, selected_idx).flatten()

In [41]:
x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]

In [42]:
print(f"선택된 클래스: {selected_classes}")
print(f"  - 학습 샘플 수: {len(x_train)}")
print(f"  - 테스트 샘플 수: {len(x_test)}\n")

선택된 클래스: ['cat', 'dog', 'horse']
  - 학습 샘플 수: 15000
  - 테스트 샘플 수: 3000



In [43]:
# 라벨을 0~2로 다시 매핑
label_map = {v: i for i, v in enumerate(selected_idx)}
y_train = np.array([label_map[y_val[0]] for y_val in y_train], dtype=np.int32)
y_test = np.array([label_map[y_val[0]] for y_val in y_test], dtype=np.int32)

# y_train과 y_test의 재매핑 후 고유 값 확인 (디버깅용)
print("Unique values in filtered y_train after remapping:", np.unique(y_train))
print("Unique values in filtered y_test after remapping:", np.unique(y_test))

Unique values in filtered y_train after remapping: [0 1 2]
Unique values in filtered y_test after remapping: [0 1 2]


In [44]:
print("Unique values in filtered y_train before remapping:", np.unique(y_train))
print("Unique values in filtered y_test before remapping:", np.unique(y_test))

Unique values in filtered y_train before remapping: [0 1 2]
Unique values in filtered y_test before remapping: [0 1 2]


In [45]:
# 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0

In [46]:
# Define the model architecture using only CNN layers for classification
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# Add a convolutional layer with 3 filters for classification
model.add(layers.Conv2D(3, (3, 3), activation='relu', padding='same')) # Add padding to maintain spatial dimensions
model.add(layers.GlobalAveragePooling2D()) # Global Average Pooling to reduce spatial dimensions
model.add(layers.Activation('softmax')) # Softmax activation for classification

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# --- 데이터 형태 및 값 확인 (디버깅용) ---
print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)
print("Sample y_train labels:", y_train[:10])
print("Sample y_test labels:", y_test[:10])
# ------------------------------------------


# Train the model
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_test, y_test))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Shape of x_train: (15000, 32, 32, 3)
Shape of y_train: (15000,)
Shape of x_test: (3000, 32, 32, 3)
Shape of y_test: (3000,)
Sample y_train labels: [2 0 2 2 0 0 0 1 0 0]
Sample y_test labels: [0 0 1 2 1 2 2 1 1 1]
Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 42ms/step - accuracy: 0.4409 - loss: 1.0305 - val_accuracy: 0.5767 - val_loss: 0.8587
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - accuracy: 0.5967 - loss: 0.8342 - val_accuracy: 0.5507 - val_loss: 0.9450
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.6439 - loss: 0.7687 - val_accuracy: 0.6650 - val_loss: 0.7254
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - accuracy: 0.6831 - loss: 0.7020 - val_accuracy: 0.6870 - val_loss: 0.6955
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step - accuracy: 0.7103 - loss: 0.6418 - val_accuracy: 0.6693 - val_loss: 0.6953
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - accuracy: 0.7342 - loss: 0.6169 - val_accuracy: 0.7300 - val_loss: 0.6288
Epoch 7

In [47]:
# Get the final training and validation metrics
final_train_accuracy = history.history['accuracy'][-1]
final_val_accuracy = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

# Print the summarized results
print(f"최종 학습 정확도: {final_train_accuracy:.4f}")
print(f"최종 검증 정확도: {final_val_accuracy:.4f}")
print(f"최종 학습 손실: {final_train_loss:.4f}")
print(f"최종 검증 손실: {final_val_loss:.4f}")

최종 학습 정확도: 0.7843
최종 검증 정확도: 0.7500
최종 학습 손실: 0.5187
최종 검증 손실: 0.5747
